# Notebook Containing the data generation, NN Modelling and Optimization Attempts for two windturbines

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Import Pyomo
from pyomo import environ
from pyomo.core import *
import pyomo.environ as pyo
import gurobipy


# import constraint Learning Tool
import sys
import os

sys.path.insert(0, os.path.join(os.getcwd(), 'DistCL_code'))
from distcl import distcl

## Load Data

In [2]:
data = pd.read_csv("data/two_windturbine.csv")
data.head()

,x_turb2,y_turb2,wind_speed,wind_direction,turbulence_intensity,turbine1_power,turbine2_powers,farm_power
0,800,0,8.0,270.0,0.06,1753.954459,615.070589,2369.025048
1,800,200,8.0,270.0,0.06,1753.954459,1751.593411,3505.547870
2,800,400,8.0,270.0,0.06,1753.954459,1753.954459,3507.908918
3,800,600,8.0,270.0,0.06,1753.954459,1753.954459,3507.908918
4,900,0,8.0,270.0,0.06,1753.954459,694.023084,2447.977544


# Train NN and generate constraints

In [3]:
### Test/Train Split
from sklearn.model_selection import train_test_split

# Split the data into training, validation, and test sets
train_data, temp_data = train_test_split(data, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

val_ind = val_data.index
test_ind = test_data.index

In [4]:
# Retrain the model with the best parameters
cl_tool = distcl(X=data[["x_turb2", "y_turb2", "wind_speed", "wind_direction"]],#, "turbulence_intensity"]],
            y=data["farm_power"], n_preds=0, val_ind=val_ind, test_ind=test_ind)

model, preds_test, sd_test, y_test = cl_tool.train(n_hidden=1, n_nodes=20, iters=5000, drop=0.05, learning_rate=1e-4)

DistFCNN(
  (lin_layers): ModuleList(
    (0): Linear(in_features=4, out_features=20, bias=True)
  )
  (output_mean_layer): Linear(in_features=20, out_features=1, bias=True)
  (output_sd_layer): Linear(in_features=20, out_features=1, bias=True)
  (droput_layers): ModuleList(
    (0): Dropout(p=0.05, inplace=False)
  )
)
cpu
epoch 0 loss 97177.2734375
epoch 500 loss 39.99507522583008
epoch 1000 loss 18.152414321899414
epoch 1500 loss 12.949406623840332
epoch 2000 loss 7.279285430908203
epoch 2500 loss 7.22467565536499
epoch 3000 loss 7.16231107711792
epoch 3500 loss 7.0245513916015625
epoch 4000 loss 6.843708038330078
epoch 4500 loss 5.9014482498168945
NN fitting process finished with a validation GAUSSIANNLL loss of 5.844993591308594 in epoch 4999


In [5]:
cons = cl_tool.constraint_build(model)
cons.to_csv('inputs/constraints_twoturbines.csv')
cons = pd.read_csv('inputs/constraints_twoturbines.csv', index_col=0)
cons

,intercept,layer,node,node_0,node_1,node_2,node_3,node_4,node_5,node_6,...,node_10,node_11,node_12,node_13,node_14,node_15,node_16,node_17,node_18,node_19
0,-0.442807,0,0,-0.003743,0.268222,-0.432792,-0.387390,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.413067,0,1,-0.192577,0.134079,0.019742,0.386540,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.396681,0,2,-0.044372,0.132306,-0.167732,-0.110929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.498764,0,3,-0.477674,-0.331141,-0.206112,0.018522,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.067214,0,4,0.197668,0.300011,-0.312961,-0.197491,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.108085,0,5,0.181609,0.415194,-0.082629,0.347293,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-0.088656,0,6,-0.080592,0.052907,0.457334,-0.458401,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.259942,0,7,-0.314769,-0.126583,-0.168095,0.396679,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.165408,0,8,-0.324090,-0.230166,-0.322284,-0.445601,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,-0.311103,0,9,-0.291870,0.429799,0.210897,0.227032,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Optimization Model



### Decision Variables
\( x \): x-coordinate of the second turbine relative to the first.
\( y \): y-coordinate of the second turbine relative to the first.

### **Objective Function**
$$ \max_{x, y} P(x, y) $$

### Constraints
$ x_{\min} \leq x \leq x_{\max} $
   
$ y_{\min} \leq y \leq y_{\max} $


In [ ]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, minimize, value

### Define the optimization model
model = ConcreteModel()

contextual_sample = data[["x_turb2", "y_turb2", "wind_speed", "wind_direction"]].iloc[[0]]
model.var_ind = pyo.Set(initialize=contextual_sample.columns.sort_values())

# Define decision variables for the x and y coordinates of turbine 2
# model.x_turb2 = Var(within=PositiveReals, bounds=(800, 5000))
# model.y_turb2 = Var(within=PositiveReals, bounds=(0, 600))

model.x = pyo.Var(model.var_ind, within=pyo.Reals)
model.y = pyo.Var(pyo.Any, dense=False, domain=pyo.Reals) # learned variables (demand)

model.power = pyo.Var(within=pyo.Reals) # saving power per scenario

# TODO: x_turb2 and y_turb2 impact the generated power and have to thus be part of model in the objective function 
# TODO: 

# obj function 
def obj_expression(model):
    return model.power
model.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

#power generation
def power(model):
    return model.power ==  model.y['power']
model.const_power = pyo.Constraint(rule=power)

#constraint for fixing contextual information
def const9_rule(model,x_ind):
    if x_ind=='x_turb2':
        return pyo.Constraint.Skip
    if x_ind=='y_turb2':
        return pyo.Constraint.Skip
    else:
        return model.x[x_ind] == contextual_sample.loc[0,x_ind]
model.const9 = pyo.Constraint(model.var_ind, rule=const9_rule)

# # range limits for x_turb2 and y_turb2
# model.x_turb2_lb = pyo.Constraint(expr=800 <= model.x['x_turb2'])   # Lower bound
# model.x_turb2_ub = pyo.Constraint(expr=model.x['x_turb2'] <= 5000)  # Upper bound

# model.y_turb2_lb = pyo.Constraint(expr=0 <= model.x['y_turb2'])     # Lower bound
# model.y_turb2_ub = pyo.Constraint(expr=model.x['y_turb2'] <= 600)   # Upper bound





# ## min distance 
# # Define constraints if any (for example, minimum distance between turbines)
# def distance_constraint(model):
#     return ((model.x_turb2 - 0)**2 + (model.y_turb2 - 0)**2) >= 400**2 
# model.distance_constraint = Constraint(rule=distance_constraint)


### Constraint Embedding
cl_tool.const_embed(opt_model=model, constaints=cons, outcome='power', deterministic = True)


# # Solve the optimization problem
# solver = SolverFactory('gurobi')
# solver.options['threads'] = 8
# solver.options['NonConvex'] = 2
# results = solver.solve(model, tee=True)




Read LP format model from file /var/folders/2j/t8d8dcsn1tg5sc8l4kz4_4ch0000gn/T/tmp6gg2mldn.pyomo.lp
Reading time = 0.00 seconds
x1: 3 rows, 4 columns, 4 nonzeros
Set parameter Threads to value 8
Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 24.4.0 24E263)

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 8 threads

Non-default parameters:
NonConvex  2
Threads  8

Optimize a model with 3 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x9d4aced9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+00, 3e+02]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Infeasible or unbounded model
model.name="unknown";
    - termination condition: infeasibleOrUnbounded
    - message from solver: Problem pro

In [30]:
model.pprint()

1 Set Declarations
    var_ind : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'wind_direction', 'wind_speed', 'x_turb2', 'y_turb2'}

5 Var Declarations
    power : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    v : Size=0, Index=Any
        Key : Lower : Value : Upper : Fixed : Stale : Domain
    v_ind : Size=0, Index=Any
        Key : Lower : Value : Upper : Fixed : Stale : Domain
    x : Size=4, Index=var_ind
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
        wind_direction :  None :  None :  None : False :  True :  Reals
            wind_speed :  None :  None :  None : False :  True :  Reals
               x_turb2 :  None :  None :  None : False :  True :  Reals
               y_turb2 :  None :  None :  None : False :  True :  Reals
    y : Size=1, Index=Any
        Key   : Low

In [25]:
list(model.x)

['wind_direction', 'wind_speed', 'x_turb2', 'y_turb2']

In [26]:
print(model.x['x_turb2'].value)

None


In [ ]:
, model.x['y_turb2'].value, model.power.value

KeyError: "Index 'x_turb2' is not valid for indexed component 'x'"

In [12]:
# After solving:
print("\nOptimization Results:")
#print(f"Objective (Maximized Power): {pyo.value(model.OBJ)}")

# Print learned variables
# print("\nLearned Variables (y):")
# for k in model.y:
#     print(f"{k}: {pyo.value(model.y[k])}")

# Print x variables (contextual features and decision variables)
print("\nContextual/Decision Variables (x):")
for k in model.x:
    print(f"{k}: {pyo.value(model.x[k])}")

# Print power directly
print(f"\nGenerated Power: {pyo.value(model.power)}")



Optimization Results:

Contextual/Decision Variables (x):
ERROR: evaluating object as numeric value: x[wind_direction,1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[wind_direction,1]


ValueError: No value for uninitialized NumericValue object x[wind_direction,1]

In [8]:
print(list(model.var_ind))


['wind_direction', 'wind_speed', 'x_turb2', 'y_turb2']


In [9]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, minimize, value

### Define the optimization model
model = ConcreteModel()

model.N = pyo.Set(initialize=[1])
model.W = pyo.Set(initialize=[1])

contextual_sample = data[["x_turb2", "y_turb2", "wind_speed", "wind_direction"]].iloc[[0]]
model.var_ind = pyo.Set(initialize=contextual_sample.columns.sort_values())

# Define decision variables for the x and y coordinates of turbine 2
# model.x_turb2 = Var(within=PositiveReals, bounds=(800, 5000))
# model.y_turb2 = Var(within=PositiveReals, bounds=(0, 600))

model.x = pyo.Var(model.var_ind, model.N, within=pyo.Reals)
model.y = pyo.Var(pyo.Any, dense=False, domain=pyo.Reals) # learned variables (demand)

model.power = pyo.Var(within=pyo.Reals) # saving profit per scenario

# TODO: x_turb2 and y_turb2 impact the generated power and have to thus be part of model in the objective function 
# TODO: 

# obj function 
def obj_expression(model):
    return model.power
model.OBJ = pyo.Objective(rule=obj_expression, sense=pyo.maximize)

#power generation
def power(model, n, w):
    return model.power ==  model.y['power', n, w]
model.const_power = pyo.Constraint(model.N, model.W, rule=power)

#constraint for fixing contextual information
def const9_rule(model,x_ind,n):
    if x_ind=='x_turb2':
        return pyo.Constraint.Skip
    if x_ind=='y_turb2':
        return pyo.Constraint.Skip
    else:
        return model.x[x_ind,n] == contextual_sample.loc[n,x_ind]
model.const9 = pyo.Constraint(model.var_ind, model.N  , rule=const9_rule)


# ## min distance 
# # Define constraints if any (for example, minimum distance between turbines)
# def distance_constraint(model):
#     return ((model.x_turb2 - 0)**2 + (model.y_turb2 - 0)**2) >= 400**2 
# model.distance_constraint = Constraint(rule=distance_constraint)




### Constraint Embedding
cl_tool.const_embed(opt_model=model, constaints=cons, outcome='power', deterministic = True)


# Solve the optimization problem
solver = SolverFactory('glpk')
solver.options['threads'] = 8
solver.options['NonConvex'] = 2
results = solver.solve(model, tee=True)




ERROR: Rule failed when generating expression for Constraint const9 with index
('wind_direction', 1): KeyError: 1
ERROR: Constructing component 'const9' from data=None failed:
        KeyError: 1


KeyError: 1

In [ ]:
contextual_sample.columns.sort_values()

Index(['wind_direction', 'wind_speed', 'x_turb2', 'y_turb2'], dtype='object')

In [ ]:
print(SolverFactory('gurobi').available())

False


In [ ]:


# Get the optimized coordinates and farm power
optimized_x_turb2 = value(model.x_turb2)
optimized_y_turb2 = value(model.y_turb2)
turbine_powers, optimized_farm_power = two_turbine_simulation(fmodel, 
                                                              x_turb2=optimized_x_turb2, 
                                                              y_turb2=optimized_y_turb2,
                                                              wind_speeds=wind_speeds, 
                                                              wind_directions=wind_directions, 
                                                              turbulence_intensities=turbulence_intensities)

print(f"Optimized x coordinate of turbine 2: {optimized_x_turb2}")
print(f"Optimized y coordinate of turbine 2: {optimized_y_turb2}")
print(f"Optimized farm power: {optimized_farm_power[0]}")